In [4]:
# Just load modules that are used later...
import numpy as np
import tensorflow as tf
from IPython.display import SVG
import matplotlib.pyplot as plt
%matplotlib inline

# Notes for Journal Club 2016/02/23

* Title:	 A Critical Review of Recurrent Neural Networks for Sequence Learning
* Authors:	Lipton, Zachary C.; Berkowitz, John; Elkan, Charles
* Publication Date:	05/2015
* URL: http://arxiv.org/abs/1506.00019
* Keywords:	Computer Science - Learning, Computer Science - Neural and Evolutionary Computing
* Abstract
> Countless learning tasks require dealing with sequential data. Image captioning, speech synthesis, and music generation all require that a model produce outputs that are sequences. In other domains, such as time series prediction, video analysis, and musical information retrieval, a model must learn from inputs that are sequences. Interactive tasks, such as translating natural language, engaging in dialogue, and controlling a robot, often demand both capabilities. Recurrent neural networks (RNNs) are connectionist models that capture the dynamics of sequences via cycles in the network of nodes. Unlike standard feedforward neural networks, recurrent networks retain a state that can represent information from an arbitrarily long context window. Although recurrent neural networks have traditionally been difficult to train, and often contain millions of parameters, recent advances in network architectures, optimization techniques, and parallel computation have enabled successful large-scale learning with them. In recent years, systems based on long short-term memory (LSTM) and bidirectional (BRNN) architectures have demonstrated ground-breaking performance on tasks as varied as image captioning, language translation, and handwriting recognition. In this survey, we review and synthesize the research that over the past three decades first yielded and then made practical these powerful learning models. When appropriate, we reconcile conflicting notation and nomenclature. Our goal is to provide a self-contained explication of the state of the art together with a historical perspective and references to primary research.



## Main topics
* Reccurent Neural Networks (RNNs)
    * LSTM (Long short-term memory)
    * Bidirectional Neuroal Networks (BRNN)
* Distributed Represantation
    * Word2Vec
* Applications
    * Language translation
    * Image Capturing

# Architectures

## Feedfoward Neural Network
<img src=FNN.png width=400px>

## Reccurent Neural Network
<img src=RNN1.png width=800px>

A verbose discription
<img src=RNN2.png width=800px>
xt-1でなくxt

 $$
 \mathbf{h}^{(t)} = \sigma(W^{hx}\mathbf{x}^{(t)} + W^{hh}\mathbf{h}^{(t-1)}) \\
 \mathbf{y}^{(t)} = \mathrm{softmax}(W^{yh}\mathbf{h}^{(t)})
 $$

## LSTM
<img src=https://qiita-image-store.s3.amazonaws.com/0/60969/91876b7a-bf2a-5585-a0bf-c6600c6d638e.png width=800px>
Retrieved from [Here](http://qiita.com/t_Signull/items/21b82be280b46f467d1b).
<img src=LSTM.png width=800px>
Fig 10: LSTM memory cell with a forget gate as  described by Gers et al. 2000

$$
\mathbf{g}^{(t)} = \phi (W^{gx} \mathbf{x}^{(t)} + W^{gh} \mathbf{h}^{(t-1)}) \\
\mathbf{i}^{(t)} = \sigma (W^{ix} \mathbf{x}^{(t)} + W^{ih} \mathbf{h}^{(t-1)}) \\
\mathbf{f}^{(t)} = \sigma (W^{fx} \mathbf{x}^{(t)} + W^{fh} \mathbf{h}^{(t-1)}) \\
\mathbf{o}^{(t)} = \sigma (W^{ox} \mathbf{x}^{(t)} + W^{oh} \mathbf{h}^{(t-1)}) \\
\mathbf{s}^{(t)} = \mathbf{g}^{(t)} \otimes \mathbf{i}^{(t)} + \mathbf{s}^{(t-1)} \otimes \mathbf{f}^{(t)}\\
\mathbf{h}^{(t)} = \phi (\mathbf{s}^{(t)}) \otimes \mathbf{o}^{(t)}
$$

A verbose discription
<img src=LSTM_verbose.png width=800px>

<img src=gates.png width=800px>

## BRNN 
<img src=BRNN.png width=800px>

# Applications

## Word2Vec (Skip-gram)

### Unigram
$P(w) = \frac{number\ of\ appearance\ of\ w}{total\ number\ of\ words}$
### Skip-gram model
* w: word
* c: context (word that appeared around w)


### Negative Sampling

## Examples
<img src=http://web.mit.edu/amarbles/www/docs/word2vec_PCA.png width=800px>
Retreived from [Here](http://web.mit.edu/amarbles/www/neuro_word2vec.html).

Following code retrived  from [Here](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/word2vec/word2vec_basic.py) (and slightly modified). For more information, see [Here](https://www.tensorflow.org/versions/r0.7/tutorials/word2vec/index.html#highlights).

ここは別ノートに移したほうがいいかも知らん...

## Natural Language Translation
### // BLEU Score

<img src=https://qiita-image-store.s3.amazonaws.com/0/100698/949b1930-5db2-008c-4db1-ad24b2eea4fe.png width=800px>

${\begin{align}
{\bf i}_n & = \tanh \bigl( W_{xi} \cdot {\bf x}_n \bigr), \\
{\bf p}_n & = {\rm LSTM} \bigl( W_{ip} \cdot {\bf i}_n + W_{pp} \cdot {\bf p}_{n-1} \bigr), \\
{\bf q}_1 & = {\rm LSTM} \bigl( W_{pq} \cdot {\bf p}_{|{\bf w}|} \bigr), \\
{\bf q}_m & = {\rm LSTM} \bigl( W_{yq} \cdot {\bf y}_{m-1} + W_{qq} \cdot {\bf q}_{m-1} \bigr), \\
{\bf j}_m & = \tanh \bigl( W_{qj} \cdot {\bf q}_m \bigr), \\
{\bf y}_m & = {\rm softmax} \bigl( W_{jy} \cdot {\bf j}_m \bigr).
\end{align}
}$

retrieved from [Here](http://qiita.com/odashi_t/items/a1be7c4964fbea6a116e). You can see Chainer implementation (Japanese).

## Image Capturing
Basically, That's translation from image to language.

In [ ]:
<img src=https://qiita-image-store.s3.amazonaws.com/0/100698/949b1930-5db2-008c-4db1-ad24b2eea4fe.png width=800px>

References
* [Tensorflow Tutorial](https://www.tensorflow.org/versions/r0.7/tutorials/index.html). Awesome
* [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
* [Goldberg 2014](http://arxiv.org/abs/1402.3722)
    * This note is an attempt to explain equation (4) (negative sampling) in "Distributed Representations of Words and Phrases and their Compositionality" by Tomas Mikolov, Ilya Sutskever, Kai Chen, Greg Corrado and Jeffrey Dean.
* [Vinyals 2014](http://arxiv.org/abs/1411.4555)
    * Show and Tell: A Neural Image Caption Generator

参考文献
* [わかるLSTM](http://qiita.com/t_Signull/items/21b82be280b46f467d1b)
* [岩波データサイエンス2](https://sites.google.com/site/iwanamidatascience/vol2/natural-language-processing)